In [3]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import numpy as np
from PIL import Image, ImageTk
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import os
from collections import OrderedDict

class ArtisticColorizationSystem:
    def __init__(self):
        # Available artistic styles
        self.styles = {
            "Van Gogh (Starry Night)": "vangogh",
            "Monet (Water Lilies)": "monet",
            "Picasso (Cubist)": "picasso",
            "Hokusai (The Great Wave)": "hokusai",
            "Rembrandt (Chiaroscuro)": "rembrandt",
            "Modern Pop Art": "popart",
            "Impressionist": "impressionist",
            "No Style (Natural Color)": "natural"
        }

        # Initialize models
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.colorization_model = None
        self.style_transfer_model = None
        self.vgg19 = None

        # GUI components
        self.root = None
        self.input_image = None
        self.output_image = None
        self.style_var = None

        self.setup_models()

    def setup_models(self):
        """Initialize the colorization and style transfer models"""
        print("Setting up models...")

        # Setup VGG19 for style extraction (simplified)
        self.vgg19 = models.vgg19(pretrained=True).features.to(self.device).eval()

        # For the demo, we'll use a simplified approach
        # In production, you would load pretrained colorization and style transfer models
        print("Models initialized (simulated for demo)")

CORE MODELS:

In [4]:
class ColorizationNet(nn.Module):
    """U-Net based architecture for colorization"""
    def __init__(self, input_channels=1, output_channels=2):
        super(ColorizationNet, self).__init__()

        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.decoder2 = nn.Sequential(
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

        self.decoder1 = nn.Sequential(
            nn.Conv2d(192, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, output_channels, kernel_size=1)
        )

    def forward(self, x):
        # Encoder path
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)

        # Bottleneck
        bottleneck = self.bottleneck(enc2)

        # Decoder path with skip connections
        dec2 = self.decoder2(torch.cat([bottleneck, enc2], dim=1))
        dec1 = self.decoder1(torch.cat([dec2, enc1], dim=1))

        return dec1

In [1]:
class StyleTransfer:
    """Apply artistic styles to colorized images"""
    @staticmethod
    def extract_features(image, model):
        """Extract features from VGG19 layers"""
        features = []
        x = image
        for i, layer in enumerate(model):
            x = layer(x)
            if i in {1, 6, 11, 20, 29}:  # Selected layers for style/content
                features.append(x)
        return features

    @staticmethod
    def gram_matrix(tensor):
        """Compute Gram matrix for style representation"""
        _, d, h, w = tensor.size()
        tensor = tensor.view(d, h * w)
        gram = torch.mm(tensor, tensor.t())
        return gram

    @staticmethod
    def apply_style(colorized_img, style_name):
        """Apply specific artistic style to image"""
        # Convert to LAB and then to RGB for processing
        if isinstance(colorized_img, np.ndarray):
            colorized_img = torch.from_numpy(colorized_img).float().to('cpu')

        # In production, this would use actual style transfer models
        # For demo, we apply simulated style effects
        img_np = colorized_img.numpy().transpose(1, 2, 0)

        # Apply different filters based on selected style
        if style_name == "vangogh":
            # Simulate Van Gogh's brush strokes
            return StyleTransfer.apply_vangogh_effect(img_np)
        elif style_name == "monet":
            # Simulate Monet's impressionism
            return StyleTransfer.apply_monet_effect(img_np)
        elif style_name == "popart":
            # Simulate pop art effect
            return StyleTransfer.apply_popart_effect(img_np)
        elif style_name == "natural":
            # No style, return original
            return img_np
        else:
            # Default mild artistic effect
            return StyleTransfer.apply_default_artistic(img_np)

    @staticmethod
    def apply_vangogh_effect(img):
        """Simulate Van Gogh's style"""
        # This would be replaced with actual neural style transfer
        # For demo, using OpenCV filters
        result = cv2.stylization(img, sigma_s=60, sigma_r=0.6)
        result = cv2.detailEnhance(result, sigma_s=10, sigma_r=0.15)
        return result

    @staticmethod
    def apply_monet_effect(img):
        """Simulate Monet's impressionist style"""
        result = cv2.edgePreservingFilter(img, flags=1, sigma_s=60, sigma_r=0.4)
        # Add warm impressionist tone
        result = cv2.applyColorMap(result, cv2.COLORMAP_SUMMER)
        return result

    @staticmethod
    def apply_popart_effect(img):
        """Apply pop art style"""
        # Posterization effect
        result = cv2.medianBlur(img, 3)
        Z = result.reshape((-1, 3))
        Z = np.float32(Z)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = 8
        _, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()].reshape((result.shape))
        return result

    @staticmethod
    def apply_default_artistic(img):
        """Default artistic enhancement"""
        result = cv2.detailEnhance(img, sigma_s=10, sigma_r=0.15)
        return result

In [5]:
def main():
    """Main application entry point"""
    print("Starting Artistic Style Transfer in Colorization System...")
    print("=" * 60)

    # Check for required packages
    try:
        import torch
        import torchvision
        import PIL
        import cv2
        import numpy as np
    except ImportError as e:
        print(f"Error: Missing required package - {e}")
        print("Please install required packages:")
        print("pip install torch torchvision pillow opencv-python numpy")
        return

    # Create and run the system
    # system = ArtisticColorizationSystem()
    # gui = ArtisticColorizationGUI(system)
    # gui.run()

# if __name__ == "__main__":
#     main()


GUI Interface


The `TclError` means that a graphical user interface (GUI) application is trying to run in a headless environment (like Colab) without a display server. To fix this, we will modify the `main` function to not initialize the `tkinter` GUI directly. If you need a graphical output, consider alternative visualization methods like displaying images directly in the notebook output using `matplotlib` or saving them to files.

In [ ]:
!pip install gradio torch torchvision Pillow opencv-python numpy


In [6]:

import gradio as gr
import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

def apply_style_effect(image, style_name):
    """Apply artistic style effects using OpenCV"""
    img_array = np.array(image)

    if style_name == "Van Gogh":
        # Stylization effect
        result = cv2.stylization(img_array, sigma_s=60, sigma_r=0.6)
    elif style_name == "Monet":
        # Impressionist effect
        result = cv2.edgePreservingFilter(img_array, flags=1, sigma_s=60, sigma_r=0.4)
    elif style_name == "Picasso":
        # Cubist-like effect
        result = cv2.detailEnhance(img_array, sigma_s=10, sigma_r=0.15)
    elif style_name == "Hokusai":
        # Woodblock print effect
        result = cv2.applyColorMap(img_array, cv2.COLORMAP_SUMMER)
    elif style_name == "Pop Art":
        # Posterization effect
        Z = img_array.reshape((-1,3))
        Z = np.float32(Z)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = 8
        _, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()].reshape((img_array.shape))
    else:  # Natural
        result = img_array

    return Image.fromarray(result)

def colorize_and_style(input_image, style_choice):
    """Main processing function"""
    # Convert to grayscale first (simulating colorization input)
    if input_image.mode != 'L':
        grayscale = input_image.convert('L')
        # Convert back to RGB for demo (simulating colorization)
        colorized = Image.merge('RGB', [grayscale, grayscale, grayscale])
    else:
        colorized = Image.merge('RGB', [input_image, input_image, input_image])

    # Apply selected style
    styled_image = apply_style_effect(colorized, style_choice)

    return styled_image

# Create Gradio interface
iface = gr.Interface(
    fn=colorize_and_style,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Dropdown(
            choices=["Van Gogh", "Monet", "Picasso", "Hokusai", "Pop Art", "Natural"],
            value="Natural",
            label="Select Artistic Style"
        )
    ],
    outputs=gr.Image(type="pil", label="Styled Output"),
    title="🎨 Artistic Style Transfer in Colorization",
    description="Upload an image and select an artistic style to apply. The system will colorize (simulated) and apply the selected style.",
    examples=[
        ["https://images.unsplash.com/photo-1506744038136-46273834b3fb", "Van Gogh"],
        ["https://images.unsplash.com/photo-1519681393784-d120267933ba", "Monet"]
    ]
)

# Launch in Colab
iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://29abe7b4071f022e9e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://29abe7b4071f022e9e.gradio.live
